<a href="https://colab.research.google.com/github/astromad/MyDeepLearningRepo/blob/master/PowerFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#!pip install sklearn

In [4]:
import numpy as np
import pandas as pd
from pandas import read_csv
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import LSTM
from keras.layers import Dense, Dropout, Activation
from sklearn.preprocessing import StandardScaler

In [5]:
path = '/content/drive/MyDrive/ColabData/Power/PowerUseThis.csv'

COLS = [4,5,6,7,8,9]
df = read_csv(path,usecols=COLS)
df.head(10)


,37,52,22,36,17,24
0,22,54,52,14,59,4
1,29,8,37,38,5,34
2,14,10,40,51,30,1
3,36,7,26,8,19,15
4,13,54,41,37,27,32
5,50,17,57,35,4,12
6,47,18,58,53,51,30
7,14,49,9,7,45,23
8,41,33,10,59,29,15
9,41,37,17,50,21,1


In [6]:
df['24'].value_counts().nlargest(n=5)

24    64
18    60
13    50
4     49
10    49
Name: 24, dtype: int64

In [7]:
scalar = StandardScaler().fit(df.values)
tf_data = scalar.transform(df.values)
tf_df = pd.DataFrame(data=tf_data, index=df.index)

In [8]:
rows = df.values.shape[0]
window =10
features = df.values.shape[1]

In [9]:
train = np.empty([rows - window, window, features],dtype = float)
label = np.empty([rows-window,features],dtype = float)
for i in range(0,rows-window):
  train[i] = tf_df.iloc[i:i+window, 0:features]
  label[i] = tf_df.iloc[i+window:i+window+1,0:features]

In [31]:
batch_size = 6
model = Sequential()
model.add(LSTM(150,activation='relu',input_shape=(window,features),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(56,activation='relu',return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(30,activation='relu',return_sequences=False))
model.add(Dropout(0.1))
model.add(Dense(features))
model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])


In [32]:
model.fit(train,label,batch_size=batch_size,epochs=100)

Epoch 1/100
231/231 [==============================] - 7s 16ms/step - loss: 1.0007 - accuracy: 0.1827
Epoch 2/100
231/231 [==============================] - 4s 17ms/step - loss: 0.9922 - accuracy: 0.2159
Epoch 3/100
231/231 [==============================] - 4s 17ms/step - loss: 0.9857 - accuracy: 0.2404
Epoch 4/100
231/231 [==============================] - 4s 18ms/step - loss: 0.9846 - accuracy: 0.2419
Epoch 5/100
231/231 [==============================] - 5s 20ms/step - loss: 0.9802 - accuracy: 0.2455
Epoch 6/100
231/231 [==============================] - 5s 20ms/step - loss: 0.9776 - accuracy: 0.2462
Epoch 7/100
231/231 [==============================] - 4s 18ms/step - loss: 0.9771 - accuracy: 0.2527
Epoch 8/100
231/231 [==============================] - 4s 18ms/step - loss: 0.9767 - accuracy: 0.2318
Epoch 9/100
231/231 [==============================] - 4s 19ms/step - loss: 0.9732 - accuracy: 0.2534
Epoch 10/100
231/231 [==============================] - 4s 19ms/step - loss: 0.968

In [35]:
to_predict= scalar.transform(df.tail(10).values)

In [36]:
output = model.predict(np.array([to_predict]))
print(scalar.inverse_transform(output).astype(int)[0])

1/1 [==============================] - 0s 393ms/step
[ 2 25 14 36 40 19]
